## Bibliotecas

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
import os
import joblib

## 1 Base de dados com dados limpos.

In [2]:
# Base Suja
carros = pd.read_csv('https://raw.githubusercontent.com/RafaelSilvax06/ProjetoIA-Car-Price-ML/refs/heads/main/data_base/car_sales_data.csv')

#Aplicando a mesma limpeza da fase 1

# Tradução de colunas
carros.columns = carros.columns = ['Fabricante', 'Modelo', 'Potencia Motor', 'Tipo Combustivel', 'Ano De Fabricacao', 'Quilometragem', 'Preco']

# Convertando preço em reais.
carros['Preco'] = carros['Preco'].astype(float)
carros['Preco'] = carros['Preco'] * 5.38

# Converter Quilometragem para float.
carros['Quilometragem'] = carros['Quilometragem'].astype(float)

# Removendo valores irreais (Preços abaixo de R$ 3.500,00)
carros_valor_irreais = carros[carros['Preco'] <= 3500.00].sort_values(by='Preco')
carros.drop(index=carros_valor_irreais.index, inplace=True)


print('Dados limpos e prontos para o pre processamento: ')
print('\n')
carros.info()

Dados limpos e prontos para o pre processamento: 


<class 'pandas.core.frame.DataFrame'>
Index: 48711 entries, 0 to 49999
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Fabricante         48711 non-null  object 
 1   Modelo             48711 non-null  object 
 2   Potencia Motor     48711 non-null  float64
 3   Tipo Combustivel   48711 non-null  object 
 4   Ano De Fabricacao  48711 non-null  int64  
 5   Quilometragem      48711 non-null  float64
 6   Preco              48711 non-null  float64
dtypes: float64(3), int64(1), object(3)
memory usage: 3.0+ MB


## 2 Definição de X e Y
Separando o conjunto de dados.


In [3]:
#Definindo X e y para criar o modelo e trabalhar na regressão.

# Y é o preço, nosso alvo.
y = carros['Preco']
# X é nossas pistas, colunas correlacionadas ao preço.
X = carros.drop('Preco', axis=1)


### 2.1 Divisão de treino com sklearn e train_test_split

In [4]:
#Usamos 80% para treino e 20% para teste.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Tamanho de X_train (treino): {X_train.shape[0]} amostras") # 80% das amostras da base de dados.
print(f"Tamanho de X_test (teste): {X_test.shape[0]} amostras") # 20% das amostras da base de dados.


Tamanho de X_train (treino): 38968 amostras
Tamanho de X_test (teste): 9743 amostras


### 2.2 Processamento de colunas.

Nosso x_train ainda tem texto e numeros misturados, vamos separo-los para fazer uma boa regressão na modelagem.

Vamos usar encoding e scaling.

Encoding:Converter colunas de texto em colunas numericas, exemplo: Toyota vira 0, BMW vira 1, etc.

Scaling: Padroniza as colunas numericas para ter medias e escalas semelhantes, assim deixando otimo o desempenho do modelo.
 

In [5]:
#Definir quais colunas são de qual tipo
colunas_categoricas = ['Fabricante', 'Modelo', 'Tipo Combustivel']
colunas_numericas = ['Potencia Motor', 'Ano De Fabricacao', 'Quilometragem']

#Criar a "esteira de processamento" (ColumnTransformer)
preprocessor = ColumnTransformer(
    transformers=[
        ('encoder', OneHotEncoder(handle_unknown='ignore'), colunas_categoricas),
        ('scaler', StandardScaler(), colunas_numericas)
    ],
    remainder='passthrough'
)

#Aplicar o processamento (TREINAR E TRANSFORMAR)
X_train_processado = preprocessor.fit_transform(X_train)

#Aplicar o processamento (APENAS TRANSFORMAR)
X_test_processado = preprocessor.transform(X_test)

#Verificar o resultado
print("Dados processados com sucesso!")
print(f"Shape de X_train_processado: {X_train_processado.shape}")
print(f"Shape de X_test_processado: {X_test_processado.shape}")

Dados processados com sucesso!
Shape de X_train_processado: (38968, 26)
Shape de X_test_processado: (9743, 26)


In [6]:
# Criar as pastas para separar os treinos e teste e transformar em aquivo .pkl para usar na modelagem
os.makedirs('../data_base/processed', exist_ok=True)

# Arquivo pickle
joblib.dump(X_train_processado, '../data_base/processed/X_train_processed.pkl')
joblib.dump(X_test_processado, '../data_base/processed/X_test_processed.pkl')
joblib.dump(y_train, '../data_base/processed/y_train.pkl')
joblib.dump(y_test, '../data_base/processed/y_test.pkl')


['../data_base/processed/y_test.pkl']